# Reference model constraint

In [1]:
# how many n soundings
# create matix with n columns
# use standard/input parameters
# allow user to input own parameters at certain location

## cell by sounding n or by coordinates??

In [2]:
#import libraries
import pandas as pd
import numpy as np
from itertools import accumulate

In [3]:
## User requirement: enter the folder location where the .dat file is saved
folder= str(r"C:/Users/u67397/AnacondaProjects/aem/input_data/")

## User requirement: enter the name of the second .dat file (without the ".dat" text)
dat_name= str("AA140005_Line1080")

## User requirement: enter the reference model conductivity specified for result file
reference_EC1= 0.001

## User requirement: enter the number of layers result file
layers= 30

## User requirement: enter the name of the second .dat file (without the ".dat" text)
con_name= str("galeisbstdem")

In [4]:
## User requirement: enter the folder location where to save files into

output_folder= str(r"C:/Users/u67397/AnacondaProjects/aem/output_data")

In [5]:
#Enter additional information. Create tests. e.g. 3 layers at xxx location with xxx EC values

In [6]:
dat= pd.read_fwf(folder+dat_name+".dat", header=None)
datlen= len(dat.index)

In [8]:
#thickness from con file
con = pd.read_csv (folder+con_name+".con", header=None) #import control file
con.replace(regex=True,to_replace=r'\t', value=r'', inplace=True)#remove tabs
tlayers=con.loc[con[0].str.contains('Thickness      = ')] #create a dataframe that contains layer thickness saved in confile (.con)
thicknessoflayers= tlayers.replace(regex=True,to_replace=r'Thickness      = ', value=r'')
thicknessoflayers=thicknessoflayers[0].str.split((' '),expand=True).replace('',np.nan)
thicknessoflayers.dropna(axis=1,inplace=True)
thicknessoflayers=thicknessoflayers.transpose().astype(float)
thicknessoflayers=np.cumsum(thicknessoflayers)
thicknessoflayers=thicknessoflayers.transpose()
thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))
thicknessoflayers=pd.DataFrame(thicknessoflayers)

In [9]:
EC=pd.Series(reference_EC1)
EC=(pd.np.tile(EC, (datlen, layers)))
EC=pd.DataFrame(EC)

In [23]:
array=np.array([[dat],[thicknessoflayers],[EC]])
array[1]

array([         0    1     2      3      4      5      6      7      8      9   \
0      3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
1      3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
2      3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
3      3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
4      3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
5      3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
6      3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
7      3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
8      3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
9      3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
10     3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
11     3.0  6.3  9.92  13.92  18.32  23.16  28.48  34.32  40.74  47.82   
12     3.0  6.3  9.92  13.92  

In [10]:
#join datafile and layer thickness dataframes
alldata=pd.concat([dat,thicknessoflayers,EC], axis=1, join='outer',ignore_index=True)
##Weighting - adjust to hit reference model. set out to side and botom x layers. 
## input constraint to relax in horiz space


In [11]:
alldata=alldata.set_index([2,3], drop=False)
alldata['all_sumcoord']=alldata[2]+alldata[3]

In [12]:
#open constraint csv
constraint_name=str("input_test") 
constraint_data= pd.read_csv(folder+constraint_name+".csv") #import control file
constraint_data['const_sumcoord']=constraint_data['x']+constraint_data['y']
constraint_data=constraint_data.set_index(['ID'], drop=False)

In [13]:
constraint_data

,ID,x,y,zt,zb,v,const_sumcoord
ID,,,,,,,
erin,erin,220955.6,6867443.6,0,-10,0.05,7088399.2
rules,rules,220956.6,6867444.6,-10,-20,0.10,7088401.2


In [14]:
matched=pd.DataFrame([0,0,0]).transpose().set_index([0])

for x in range(len(constraint_data)):
    g=(constraint_data.ID[x])
    h=(constraint_data.const_sumcoord[x])
    alldata[str(g)]= np.abs(alldata.all_sumcoord-h) 
    matched1=np.argmin(alldata[str(g)])
    e_matched=matched1[0]
    n_matched=matched1[1]
    matched2=pd.DataFrame([g,float(e_matched),float(n_matched)]).transpose().set_index([0]).astype(float)
    matched=pd.concat([matched,matched2], axis=0)

matched=matched.rename(columns={1:'E_matched',2:'N_matched'}).drop([0])
matched

,E_matched,N_matched
0,,
erin,220955.6,6867443.6
rules,220958.6,6867443.7


In [15]:
constraint_data2= pd.concat([constraint_data, matched],axis=1)
constraint_data2
constraint_data2=constraint_data2.set_index(['E_matched','N_matched'], drop=False)
constraint_data2

,,ID,x,y,zt,zb,v,const_sumcoord,E_matched,N_matched
E_matched,N_matched,,,,,,,,,
220955.6,6867443.6,erin,220955.6,6867443.6,0,-10,0.05,7088399.2,220955.6,6867443.6
220958.6,6867443.7,rules,220956.6,6867444.6,-10,-20,0.10,7088401.2,220958.6,6867443.7


In [16]:
#create and save a .csv that contains depth of investigation data for input into Discover PA software
alldata.to_csv(output_folder+'/depth_data.csv') #save doi data as csv

In [ ]:
# for row in constraint_data:
#     print(row)
    #sample[str(row)]= 0

#print(sample)
    
    
#     print('loop1_start')
#     print(row)
#     print('loop1_ends')
#     for blah in constraint_data['const_sumcoord']:
#         print('loop2_starts')
#         print (blah)
#         print('loop2_ends')
# #         while x>0:
#         for rah in sample:
#             sample[str(row)]=np.abs(test.all_sumcoord-blah)
# #                 x=x-1
#             print(sample)
                

In [ ]:
## find coordinate
## enter layer thickness (e.g 3)
## extrapolate between the points to maximum layers (e.g. 30)

In [ ]:
# #create a dataframe that contains layer thickness saved in confile (.con)
# tlayers=con.loc[con[0].str.contains('Thickness      = ')]
# thicknessoflayers= tlayers.replace(regex=True,to_replace=r'Thickness      = ', value=r'')
# thicknessoflayers=thicknessoflayers[0].str.split((' '),expand=True).replace('',np.nan)
# thicknessoflayers.dropna(axis=1,inplace=True)
# thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))
# #thicknessoflayers=pd.DataFrame(thicknessoflayers)
# thicknessoflayers

In [ ]:
# thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))


In [ ]:
def coord(easting,northing):
    easting=int(easting)
    northing=int(northing)
    print(type(easting))

coord('1','2')

In [ ]:
def print_text():
    print("complete")

In [ ]:
a = [[1, 2, 3, 4], [5, 6], [7, 8, 9]]
for i in range(len(a)):
    for j in range(len(a[i])):
        print(a[i][j], end=' ')
    print()